In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
import warnings
warnings.filterwarnings("ignore")

C:\Users\vinay\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df = pd.read_csv("SNLI_Corpus/snli_1.0_train.csv", nrows=100000)
valid_df = pd.read_csv("SNLI_Corpus/snli_1.0_dev.csv")
test_df = pd.read_csv("SNLI_Corpus/snli_1.0_test.csv")

In [3]:
train_df.tail()

,similarity,sentence1,sentence2
99995,entailment,People with costumes are gathered in a wooded ...,people wear costumes
99996,contradiction,A girl with a black dress and big white bow st...,a man wearing high heels
99997,entailment,A girl with a black dress and big white bow st...,a girl standing
99998,neutral,A girl with a black dress and big white bow st...,a girl getting ready for photo shoot
99999,entailment,A man strikes a pose on a dock with a cruise s...,A human striking a pose.


In [4]:
valid_df.tail()

,similarity,sentence1,sentence2
9995,-,A small girl wearing a pink jacket is riding o...,The girl is sitting on a carved horse made of ...
9996,contradiction,A small girl wearing a pink jacket is riding o...,The girl is moving at the speed of light.
9997,entailment,A young girl with blue and pink ribbons in her...,People in a water fountain
9998,contradiction,A young girl with blue and pink ribbons in her...,A young girl knits a sweater
9999,neutral,A young girl with blue and pink ribbons in her...,A young mother tries to grab her topless child...


In [5]:
test_df.tail()

,similarity,sentence1,sentence2
9995,contradiction,Two women are observing something together.,Two women are standing with their eyes closed.
9996,entailment,Two women are observing something together.,Two girls are looking at something.
9997,contradiction,A man in a black leather jacket and a book in ...,A man is flying a kite.
9998,entailment,A man in a black leather jacket and a book in ...,A man is speaking in a classroom.
9999,neutral,A man in a black leather jacket and a book in ...,A man is teaching science in a classroom.


In [6]:
max_length = 128 # MAximu length of input sentence to the model.
batch_size = 32
epochs = 2

# Labels in our dataset 
labels = ["contradiction", "entailment", "neutral"]

In [7]:
# Shape of the data
print(f"Total train samples : {train_df.shape[0]}")
print(f"Total validation samples: {valid_df.shape[0]}")
print(f"Total test samples: {valid_df.shape[0]}")

Total train samples : 100000
Total validation samples: 10000
Total test samples: 10000


**Dataset Overview:**

- sentence1: The premise caption that was supplied to the author of the pair.
- sentence2: The hypothesis caption that was written by the author of the pair.
- similarity: This is the label chosen by the majority of annotators. Where no majority exists, the label "-" is used (we will skip such samples here).

Here are the "similarity" label values in our dataset:

- Contradiction: The sentences share no similarity.
- Entailment: The sentences have similar meaning.
- Neutral: The sentences are neutral.

Dataset source - https://nlp.stanford.edu/projects/snli/

#### Let's take a look at all three types of similarities in sentences

In [8]:
print(f"Sentence1: {train_df.loc[1, 'sentence1']}")
print(f"Sentence2: {train_df.loc[1, 'sentence2']}")
print(f"Similarity: {train_df.loc[1, 'similarity']}")

Sentence1: A person on a horse jumps over a broken down airplane.
Sentence2: A person is at a diner, ordering an omelette.
Similarity: contradiction


In [9]:
print(f"Sentence1: {train_df.loc[2, 'sentence1']}")
print(f"Sentence2: {train_df.loc[2, 'sentence2']}")
print(f"Similarity: {train_df.loc[2, 'similarity']}")

Sentence1: A person on a horse jumps over a broken down airplane.
Sentence2: A person is outdoors, on a horse.
Similarity: entailment


In [10]:
print(f"Sentence1: {train_df.loc[3, 'sentence1']}")
print(f"Sentence2: {train_df.loc[3, 'sentence2']}")
print(f"Similarity: {train_df.loc[3, 'similarity']}")

Sentence1: Children smiling and waving at camera
Sentence2: They are smiling at their parents
Similarity: neutral


## Preprocessing

In [11]:
# We have some NaN entries in our train data, we will simply drop them.
print("Number of missing values in train_df")
print(train_df.isnull().sum())
train_df.dropna(axis=0, inplace=True)

Number of missing values in train_df
similarity    0
sentence1     0
sentence2     3
dtype: int64


In [12]:
# We have some NaN entries in our train data, we will simply drop them.
print("Number of missing values in test_df")
print(test_df.isnull().sum())
test_df.dropna(axis=0, inplace=True)

Number of missing values in test_df
similarity    0
sentence1     0
sentence2     0
dtype: int64


In [13]:
# We have some NaN entries in our train data, we will simply drop them.
print("Number of missing values in valid_df")
print(valid_df.isnull().sum())
valid_df.dropna(axis=0, inplace=True)

Number of missing values in valid_df
similarity    0
sentence1     0
sentence2     0
dtype: int64


### Distribution of Training targets

In [14]:
print("Train Target Distribution")
print(train_df.similarity.value_counts())

Train Target Distribution
entailment       33384
contradiction    33310
neutral          33193
-                  110
Name: similarity, dtype: int64


### Distribution of Validation targets

In [15]:
print("Valid Target Distribution")
print(valid_df.similarity.value_counts())

Valid Target Distribution
entailment       3329
contradiction    3278
neutral          3235
-                 158
Name: similarity, dtype: int64


### Distribution of Testing targets

In [16]:
print("Test Target Distribution")
print(test_df.similarity.value_counts())

Test Target Distribution
entailment       3368
contradiction    3237
neutral          3219
-                 176
Name: similarity, dtype: int64


#### It looks like "-" is there as a lable in train and valid data so we'll skip those samples but we'll keep in test set to generalize teh model

In [17]:
train_df = (
    train_df[train_df.similarity != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)
valid_df = (
    valid_df[valid_df.similarity != "-"]
    .sample(frac=1.0, random_state=42)
    .reset_index(drop=True)
)

### One hot encoding training, validation, test labels

In [18]:
train_df["label"] = train_df["similarity"].apply(lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2)
y_train = tf.keras.utils.to_categorical(train_df.label, num_classes = 3)

valid_df["label"] = valid_df["similarity"].apply(lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2)
y_valid = tf.keras.utils.to_categorical(valid_df.label, num_classes = 3)

test_df["label"] = test_df["similarity"].apply(lambda x: 0 if x == "contradiction" else 1 if x == "entailment" else 2)
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes = 3)

## Custom Data Loader

In [19]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

## Build the Model

In [20]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x000001CBFAD65C90>
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',    

### Create train and validation data generators

In [21]:
train_data = BertSemanticDataGenerator(
    train_df[["sentence1", "sentence2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[["sentence1", "sentence2"]].values.astype("str"),
    y_valid,
    batch_size=batch_size,
    shuffle=False,
)

## Train the Model


In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch 1/2
  20/3121 [..............................] - ETA: 3:46:16 - loss: 1.1465 - acc: 0.3547

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 111/3121 [>.............................] - ETA: 3:46:51 - loss: 1.0743 - acc: 0.4276

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

 707/3121 [=====>........................] - ETA: 3:07:37 - loss: 0.5988 - acc: 0.7572

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 720/3121 [=====>........................] - ETA: 3:06:35 - loss: 0.5990 - acc: 0.7572

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


 779/3121 [======>.......................] - ETA: 3:01:43 - loss: 0.6018 - acc: 0.7557